# Soiaporn model

Prototype the Metropolis-within-Gibbs sampling technique presented in Soiaporn et al. 2012.

The full conditionals:

$$
F_T | f, \lambda, D \sim Gamma \bigg(N_C +1, \frac{1}{1/s + (1 - f)\epsilon_0 + f \sum_{k \geq 1} w_k \epsilon_k}\bigg)
$$

$$
P(\lambda_i | F_T, f, D) \propto \frac{f_{\lambda_i}}{\epsilon_{\lambda_i}} h_{\lambda_i}
$$

Where $h_{j} = (1 - f) \epsilon_0$ if $j = 0$ and $h_j = fw_j\epsilon_j$ if $j \geq 1$.

$$
P(f | \lambda, F_T, D) \propto e^{-F_T [  (1 - f)\epsilon_0  + f \sum_{k \geq 1} \epsilon_k w_k] } \times (1-f)^{m_0(\lambda) + b - 1}f^{N_C - m_0(\lambda)+a-1}
$$


$F_T$ and $\lambda$ are sampled directly from the gamma and multinomial distributions. $f$ is sampled using a random walk Metropolis algorithm with Gaussian proposals centred on the current value of $f$. The variance of the proposal distribution was tuned to give an acceptance rate of 25%.

$\kappa$ is treated specially, they consider a logarithmically spaced grid of values to condition on. So, treat $\kappa$ as fixed. 

In [11]:
import numpy as np
import pystan
from fancy import Data

In [2]:
def scale_F_T(s, f, eps, w):
    sum_term = 0
    for k in range(len(eps)) + 1:
        sum_term += w[k] * eps[k] 
    denom = (1 + s) + ((1 - f) * eps[0]) + (f * sum_term)
    return 1/ denom

def prob_lam_i(k, F_T, f, kappa, kappa_c, eps, w, varpi, d):
    if i == 0:
        h = (1 - f) * eps[0]
    else:
        h = f * w[k] * eps[k]
    term1 = fik(kappa, kappa_c, d, varpi[i]) / eps[i]
    return term1 * h

def fik(kappa, kappa_c, d, varpi, A, theta_i):
    term1 = kappa * kappa_c / (4 * sinh(kappa) * sinh(kappa_c))
    inner = np.linalg.norm((kappa_c * d) + (kappa * varpi))
    term2 = sinh(inner) / inner
    return A * np.cos(theta) * term1 * term2

def p_f(F_T, f, eps, lam, N_C, a, b):
    sum_term = 0
    for k in range(len(eps)) + 1:
        sum_term += w[k] * eps[k]
    inner = -F_T * ((1 - f) * eps[0] + f * sum_term)
    term1 = np.exp(inner)
    m_0 = (lam == 0).sum()
    term2 = (1 - f)**(m_0 + b - 1) * f**(N_C - m_0 + a - 1)
    return term1 * term2

## Set up

In [3]:
# choose initial values
kappa = 100
kappa_c = 1000
F_T = 0.2 
f = 0.5

In [8]:
# load data
uhecr_file = '../uhecr_model/data/auger_UHECR_list.dat'
source_file = 'data/agn_catalog.dat'

data = Data()
data.add_source(source_file, 'AGN')
data.add_uhecr(uhecr_file, 'Auger2010')

In [17]:
# integral tables
sim_table = 'data/tablekappa100.data.R'
eps = pystan.read_rdump(sim_table)['table'][0]

## Sampler